# Donkey Car IMU Model

Build and run a Keras model copied from Donkey Car.

* The IMU model has been modified to accept larger telemetry inputs

In [1]:
## Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
# import tensorflow_addons as tfa
import time
import pickle
import sklearn.metrics as metrics
from os import remove
from os.path import exists

from modeling_methods import run_model, plot_metrics, save_model, create_donkey_vimu, create_normed_donkey_vimu

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from tensorflow.keras.metrics import MAE, MSE, RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow_addons.optimizers import AdamW, RectifiedAdam

2022-05-09 01:03:48.976839: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

## Directories/Variables

In [3]:
## Fill in your own here
dataset_directory = '../data/datasets/05_09_2022/00_47_15'
dropbox_directory = '/home/grant/Dropbox/Projects/DonkeyCar' # None if not used

In [4]:
seed = 1337
tf.random.set_seed(seed)
np.random.seed(seed)

In [5]:
scalers = {
    'minmax': MinMaxScaler,
    'robust': RobustScaler,
    'standard': StandardScaler,
}
models = {
    'vimu': create_donkey_vimu,
    'normed_vimu': create_normed_donkey_vimu
}

optimizers = {
    'Adam': Adam(learning_rate=0.0001, amsgrad=True),
    # 'AdamW': AdamW(amsgrad=True), # default learning rate (0.001)
    'Nadam': Nadam(learning_rate=0.0001),
    'Radam': RectifiedAdam(learning_rate=0.0001), # default learning rate: 0.001
    'RMSProp': RMSprop()
}

In [6]:
## Directories
model_directory = f'../models'

if dropbox_directory:
    dropbox_model_directory = f'{dropbox_directory}/models'
    
## File paths
cam_input_dataset_file = f'{dataset_directory}/X_img.npy'
telem_input_dataset_file = f'{dataset_directory}/X_telem.pkl'
target_dataset_file = f'{dataset_directory}/y.npy'

## Parameters
model_type = 'vimu'
scaler_type = 'robust' # minmax/robust/standard 
optimizer_type = 'Adam'

batch_sizes = [1024, 512, 256, 128, 64, 32, 16, 8, 4, 2, 1]
# batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
# batch_sizes = [1024, 512, 256, 128,] # 64, 32, 16, 8, 4]
epochs = 2000
early_stop_patience = 200 # None for no stop
dual_outputs = False
plot_results = False
verbose = 1 # 0, 1, 2

In [7]:
create_model = models[model_type]
optimizer = optimizers[optimizer_type]

## Data

### Load Datasets

In [8]:
## Load the datasets
X_cam = np.load(cam_input_dataset_file, allow_pickle=True)#.astype('uint8')

## Load telemetry as df to grab telemetry column names
telem_df = pd.read_pickle(telem_input_dataset_file).copy()
telemetry_columns = telem_df.columns
print(f'Telemetry columns: {telemetry_columns}')
## Convert to numpy, delete df
X_telem = telem_df.to_numpy() #.astype('float32')

telem_df = None
del telem_df

## Load targets
y = np.load(target_dataset_file, mmap_mode='r')

## Check Shape
print(f'{X_cam.shape = }')

Telemetry columns: Index(['speed', 'pitch', 'yaw', 'roll', 'first_lap'], dtype='object')
X_cam.shape = (100000, 120, 160, 1)


### Train-Test Split

In [9]:
if dual_outputs:
    # steering = y[:, 0], throttle = y[:, 1]
    datasets = train_test_split(X_cam, X_telem, y[:, 0], y[:, 1], test_size=0.2, random_state=seed)
else:
    datasets = train_test_split(X_cam, X_telem, y, test_size=0.2, random_state=seed)

X_telem = None
X_cam = None
y = None
del X_telem
del X_cam
del y
    
X_cam_train = datasets[0]
X_cam_test = datasets[1]
X_telem_train = datasets[2]
X_telem_test = datasets[3]
    
if dual_outputs:
    y_st_train = datasets[4]
    y_st_test = datasets[5]
    y_th_train = datasets[6]
    y_th_test = datasets[7]
else:
    y_train = datasets[4]
    y_test = datasets[5]
    
datasets = None
del datasets

### Pre-Scaler

In [10]:
if scaler_type:

    scaler_file = f'{scaler_type}_scaler_{time.strftime("%m_%d_%H_%M")}.pkl'
    scaler_path = f'../scalers/{scaler_file}'

    if dropbox_directory:
        dropbox_scaler_path = f'{dropbox_directory}/scalers/{scaler_file}'

    sc = scalers[scaler_type]()

    sc.fit(X_telem_train)
    X_telem_train = sc.fit_transform(X_telem_train)
    X_telem_test = sc.transform(X_telem_test)

    ## Save scaler as a pickle
    pickle.dump(sc, open(scaler_path, 'wb'))

    if dropbox_directory:
        pickle.dump(sc, open(dropbox_scaler_path, 'wb'))

    ## Print path
    print(f'{scaler_file = }')

scaler_file = 'robust_scaler_05_09_01_03.pkl'


### Get Input Shape(s)

In [11]:
# for i in range(len(X_cam_train)):
#     X_cam_train[i] = np.asarray(X_cam_train[i], dtype='uint8')

In [12]:
X_cam_train[0].shape

(120, 160, 1)

In [13]:
X_telem_train[0]

array([-9.74049934e-01,  2.32142997e-04,  9.17855359e-02, -9.78644080e-01,
        0.00000000e+00])

In [14]:
## Create variables
img_input_shape = X_cam_train[0].shape
tel_input_shape = X_telem_train[0].shape

## Check input shapes
print(f'{img_input_shape=}')
print(f'{tel_input_shape=}')

img_input_shape=(120, 160, 1)
tel_input_shape=(5,)


### Convert Numpy Arrays to Tensors
If nothing else, this keeps the info and warning messages from cluttering the training output.

In [15]:
X_cam_train = tf.constant(X_cam_train.astype('uint8')) #.astype('uint8'))
X_telem_train = tf.constant(X_telem_train) #.astype('float32')) #.astype('float32')
X_cam_test = tf.constant(X_cam_test.astype('uint8')) #.astype('uint8'))
X_telem_test = tf.constant(X_telem_test) #.astype('float32')) #.astype('float32')
   
    
# if dual_outputs:
    
#     y_st_train_tensor = tf.constant(y_st_train) # .astype('float32'))
#     y_st_test_tensor = tf.constant(y_st_test) #.astype('float32'))
#     y_th_train_tensor = tf.constant(y_th_train) #.astype('float32'))
#     y_th_test_tensor = tf.constant(y_th_test) #.astype('float32'))
  
#     y_train = (y_st_train_tensor, y_th_train_tensor)
#     y_test = (y_st_test_tensor, y_th_test_tensor)

#     del [y_st_train, y_st_test, y_th_train, y_th_test]    
    
# else:

#     y_train = tf.constant(y_train) #.astype('float32')
#     y_test = tf.constant(y_test) #.astype('float32')

2022-05-09 01:03:52.693253: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-09 01:03:52.725344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 01:03:52.725594: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.797GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2022-05-09 01:03:52.725613: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-09 01:03:52.729022: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-09 01:03:52.729080: I tensorflow/stream_executor

### Restore Interrupted Model

In [16]:
# from tensorflow.keras.models import Model, load_model

# checkpoint_filepath = '../models/checkpoint'
# best_model = load_model(checkpoint_filepath)
# best_results = best_model.evaluate((X_cam_test, X_telem_test_sc), y_test, batch_size=1, return_dict=True)
# # shutil.rmtree(checkpoint_filepath)
# best_model

# model_file = save_model(model_directory=model_directory, 
#                         model=best_model, 
#                         results=best_results, 
#                         batch_size=1, 
#                         dual_outputs=dual_outputs, 
#                         scaler_file=scaler_file, 
#                         telemetry_columns=telemetry_columns,
#                         dataset_directory=dataset_directory)
# model_file

## Model Loop

In [17]:
## Run models for each batch size
# print('---')
for batch_size in batch_sizes:
    print(f'Batch size {batch_size} start: {time.strftime("%H:%M:%S")}')
    model = create_model(img_input_shape=img_input_shape,
                         tel_input_shape=tel_input_shape, 
                         dual_outputs=dual_outputs,
                         x_train=X_telem_train,
                         batch_size=batch_size) 
    model.compile(loss='mse', 
                  optimizer=optimizer,
                  metrics=[RootMeanSquaredError(), 'mae'])
    model, metrics, results = run_model(model=model,
                                        X_train=(X_cam_train, X_telem_train), 
                                        y_train=y_train, 
                                        X_test=(X_cam_test, X_telem_test), 
                                        y_test=y_test, 
                                        batch_size=batch_size, 
                                        epochs=epochs,
                                        early_stop_patience=early_stop_patience,
                                        verbose=verbose)
    model_file = save_model(model_directory=model_directory, 
                            model=model, 
                            results=metrics, 
                            batch_size=batch_size, 
                            dual_outputs=dual_outputs, 
                            # scaler_file=scaler_file, 
                            telemetry_columns=telemetry_columns,
                            dataset_directory=dataset_directory,
                            dropbox_model_directory=dropbox_model_directory)
    history = {k: v for k, v in results.history.items()}
    print(f'Batch size {batch_size} end:   {time.strftime("%H:%M:%S")}')
    print(f'Epochs run: {len(history["loss"])}')
    print(f'model: {model_file}')
    print('---')
    if plot_results:
        plot_metrics(history=history, 
                     batch_size=batch_size, 
                     dual_outputs=dual_outputs)
    history = None
    model = None
    results = None
    del history
    del model
    del results
    # K.clear_session()

Batch size 1024 start: 01:03:54


2022-05-09 01:03:54.467488: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-09 01:03:54.487206: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3397935000 Hz


Epoch 1/2000


2022-05-09 01:03:55.257239: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-05-09 01:03:55.498061: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2022-05-09 01:03:55.530104: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-09 01:03:56.058754: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


21/79 [======>.......................] - ETA: 14s - loss: 0.4705 - root_mean_squared_error: 0.6860 - mae: 0.5622

KeyboardInterrupt: 